# Project 2: Extract, Transform, and Load
### Team 3: Chris Schultz and Glen Dagger

In [1]:
# Import dependencies
import pandas as pd
from census import Census
from config import api_key
from sqlalchemy import create_engine, inspect

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import county_web_scrape
import time

# Extract and Transform Data

## Beers

In [3]:
# Import beers.csv as dataframe
beers_df = pd.read_csv('./Resources/beers.csv', index_col=[0])

# Filter dataframe to desired columns
beers_cleaned_df = beers_df[['id','name','style','brewery_id','abv']]

# Display first 5 rows
beers_cleaned_df.head()

,id,name,style,brewery_id,abv
0,1436,Pub Beer,American Pale Lager,408,0.050
1,2265,Devil's Cup,American Pale Ale (APA),177,0.066
2,2264,Rise of the Phoenix,American IPA,177,0.071
3,2263,Sinister,American Double / Imperial IPA,177,0.090
4,2262,Sex and Candy,American IPA,177,0.075


## Breweries

In [4]:
# Import breweries.csv as dataframe
breweries_df = pd.read_csv('./Resources/breweries.csv')

# Strip whitespace from state column
breweries_df['state'] = breweries_df['state'].str.strip()

# Strip whitespace from name column
breweries_df['name'] = breweries_df['name'].str.strip()

# Clean dataframe
breweries_cleaned_df = breweries_df[['name', 'city', 'state']]

# Reset and rename index as id
breweries_cleaned_df.reset_index(inplace=True)
breweries_cleaned_df = breweries_cleaned_df.rename(columns = {'index':'brewery_id'})

# Display first 5 rows
breweries_cleaned_df.head()

,brewery_id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [26]:
# Export cleaned breweries table
breweries_cleaned_df.to_csv('./CleanedCSVs/breweries_table.csv', index=False)

,Unnamed: 0,city,state,county
0,0,Abingdon,VA,Washington County
1,1,Abita Springs,LA,St. Tammany Parish
2,2,Ada,MI,NaN
3,3,Afton,VA,NaN
4,4,Airway Heights,WA,Spokane County


In [6]:
# Run county_web_scrape to find county for each city in city_df and add column to dataframe
# city_df = county_web_scrape.main()

city_df = pd.read_csv('./Resources/county_list_df.csv')
city_df.head()

,Unnamed: 0,city,state,county
0,0,Abingdon,VA,Washington County
1,1,Abita Springs,LA,St. Tammany Parish
2,2,Ada,MI,NaN
3,3,Afton,VA,NaN
4,4,Airway Heights,WA,Spokane County


In [7]:
# Merge county data into breweries dataframe
breweries_counties_df = breweries_cleaned_df.merge(city_df, how='left', on=['city','state'])

# Filter columns
breweries_counties_df = breweries_counties_df[['brewery_id', 'name', 'city', 'state', 'county']]

# Display first 5 rows
breweries_counties_df

,brewery_id,name,city,state,county
0,0,NorthGate Brewing,Minneapolis,MN,Hennepin County
1,1,Against the Grain Brewery,Louisville,KY,Jefferson County
2,2,Jack's Abby Craft Lagers,Framingham,MA,Middlesex County
3,3,Mike Hess Brewing Company,San Diego,CA,San Diego County
4,4,Fort Point Beer Company,San Francisco,CA,San Francisco County
...,...,...,...,...,...
553,553,Covington Brewhouse,Covington,LA,St. Tammany Parish
554,554,Dave's Brewfarm,Wilson,WI,St. Croix County
555,555,Ukiah Brewing Company,Ukiah,CA,Mendocino County
556,556,Butternuts Beer and Ale,Garrattsville,NY,NaN


## Census Data

### County

In [9]:
# Create Census object with Census API key from the selected year
c = Census(api_key, year=2021)

# Run Census Search to retrieve income data by county
county_census_data = c.acs1.get(("NAME", "B01003_001E", "B19013_001E","B19301_001E", "B01002_001E"), {'for': 'county:*'})

# Convert to DataFrame
county_census_df = pd.DataFrame(county_census_data)

# Rename columns
county_census_cleaned_df = county_census_df.rename(columns={"B01003_001E": "population",
                                      "B19013_001E": "med_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B01002_001E": "median_age",
                                      "state": "state_code",
                                      "NAME": "county",
                                      "county": "county_code"
                                    }
                                    )

# Cast population column as integer
county_census_cleaned_df['population'] = county_census_cleaned_df['population'].astype(int)

# Split county column into separate county and state columns
county_census_cleaned_df[['county','state']] = county_census_cleaned_df.county.str.split(', ', expand=True)

# Display first 5 rows
county_census_cleaned_df.head()


,county,population,med_household_income,per_capita_income,median_age,state_code,county_code,state
0,Baldwin County,239294,63866.0,35824.0,43.9,01,003,Alabama
1,Calhoun County,115972,46524.0,24804.0,40.2,01,015,Alabama
2,Cullman County,89496,55517.0,28024.0,40.5,01,043,Alabama
3,DeKalb County,71813,41800.0,25633.0,40.2,01,049,Alabama
4,Elmore County,89304,59032.0,28515.0,40.0,01,051,Alabama


In [12]:
# Create dictionary for converting state names to abbreviations
state_abbreviations = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [13]:
# Convert State names to state abbreviations
county_census_cleaned_df.replace({"state": state_abbreviations}, inplace=True)

# Export dataframe to CSV to more easily inspect final dataset
county_census_cleaned_df.to_csv('./Resources/county_census_data.csv')

# Display first 5 rows
county_census_cleaned_df.head()

,county,population,med_household_income,per_capita_income,median_age,state_code,county_code,state
0,Baldwin County,239294,63866.0,35824.0,43.9,01,003,AL
1,Calhoun County,115972,46524.0,24804.0,40.2,01,015,AL
2,Cullman County,89496,55517.0,28024.0,40.5,01,043,AL
3,DeKalb County,71813,41800.0,25633.0,40.2,01,049,AL
4,Elmore County,89304,59032.0,28515.0,40.0,01,051,AL


In [14]:
# Merge county names into brewery 
county_brewery_combined_df = breweries_counties_df.merge(county_census_cleaned_df, how='inner', on=['county','state'])

county_brewery_combined_df.head()

,brewery_id,name,city,state,county,population,med_household_income,per_capita_income,median_age,state_code,county_code
0,0,NorthGate Brewing,Minneapolis,MN,Hennepin County,1267416,84244.0,50164.0,37.4,27,053
1,61,Surly Brewing Company,Brooklyn Center,MN,Hennepin County,1267416,84244.0,50164.0,37.4,27,053
2,258,Bauhaus Brew Labs,Minneapolis,MN,Hennepin County,1267416,84244.0,50164.0,37.4,27,053
3,277,Indeed Brewing Company,Minneapolis,MN,Hennepin County,1267416,84244.0,50164.0,37.4,27,053
4,474,Tonka Beer Company,Minnetonka,MN,Hennepin County,1267416,84244.0,50164.0,37.4,27,053


### Create Census Table by State

In [15]:
# Create Census object with Census API key from the selected year
c = Census(api_key, year=2021)

# Run Census Search to retrieve income data by county
census_data = c.acs1.get(("NAME", "B01003_001E", "B19013_001E","B19301_001E", "B01002_001E"), {'for': 'state:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Rename columns
census_cleaned_df = census_df.rename(columns={"B01003_001E": "population",
                                      "B19013_001E": "med_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B01002_001E": "median_age",
                                      "state": "state_code",
                                      "NAME": "state"
                                    }
                                    )

# Cast population column as int
census_cleaned_df['population'] = census_cleaned_df['population'].astype(int)

# Display first 5 rows
census_cleaned_df.head()

,state,population,med_household_income,per_capita_income,median_age,state_code
0,Alabama,5039877,53913.0,30608.0,39.8,01
1,Puerto Rico,3263584,22237.0,14468.0,44.1,72
2,Arizona,7276316,69056.0,36295.0,38.6,04
3,Arkansas,3025891,52528.0,29252.0,38.5,05
4,California,39237836,84907.0,42396.0,37.6,06


In [16]:
# Convert State names to state abbreviations
census_cleaned_df.replace({"state": state_abbreviations}, inplace=True)

# Export dataframe to CSV to more easily inspect final dataset
census_cleaned_df.to_csv('./Resources/census_data.csv')

# Display first 5 rows
census_cleaned_df.head()

,state,population,med_household_income,per_capita_income,median_age,state_code
0,AL,5039877,53913.0,30608.0,39.8,01
1,PR,3263584,22237.0,14468.0,44.1,72
2,AZ,7276316,69056.0,36295.0,38.6,04
3,AR,3025891,52528.0,29252.0,38.5,05
4,CA,39237836,84907.0,42396.0,37.6,06


# Load

### Connect to local database

In [157]:
# Create connection to postgresql database
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'beer_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [158]:
# Check table names using inspector
inspector = inspect(engine)
table_names = inspector.get_table_names()

table_names

['census_data', 'breweries', 'beers']

In [159]:
# Use pandas to load csv converted DataFrames into database

# Census table
census_df.to_sql(name='census_data', con=engine, if_exists='append', index=False)

# Breweries table
breweries_cleaned_df.to_sql(name='breweries', con=engine, if_exists='append', index=False)

# Beers tables
beers_cleaned_df.to_sql(name='beers', con=engine, if_exists='append', index=False)